In [1]:
from kafka import KafkaProducer
from bs4 import BeautifulSoup
import requests

In [2]:
def connect_kafka_producer():
    producer = None
    try:
        producer = KafkaProducer(bootstrap_servers=['localhost:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka', producer)
    finally:
        return producer

In [3]:
PRODUCER = connect_kafka_producer()

In [4]:
def publish_message(topic_name, key, value):
    try:
        key =  bytes(key, encoding='utf-8')
        value = bytes(value, encoding='utf-8')
        
        PRODUCER.send(topic_name, key=key, value=value)
        PRODUCER.flush()
        print('Published: {} {}'.format(key, value.decode('utf-8')))
    except Exception as ex:
        print(str(ex))

In [5]:
def get(url):
    res = requests.get(url)
    if res.status_code == 200:
        return res.text.strip()
    else:
        return ''

In [6]:
def parse(html, select):
    parser = BeautifulSoup(html, 'html.parser')
    links = parser.select(select)
    return links

In [7]:
def scrape_recipes(url):    
    html = get(url)
    links = [a['href'] for a in parse(html, select='.fixed-recipe-card__h3 a')]
    
    nutrs = []
    for link in links:
        soup = BeautifulSoup(get(link), 'html.parser')
        maybe_cal = soup.find(itemprop='calories')
        if maybe_cal is not None:
            cal = maybe_cal.get_text().split(' ')[0]
            if int(cal) < 400:
                publish_message('recipe', 'parsed', link)
            
    return nutrs

In [8]:
recipes = scrape_recipes('https://www.allrecipes.com/recipes/23014/world-cuisine/asian/thai/desserts')